## DATA CHECK NOTES

Forgive the mess! This repo has a lot of extra files -- especially data files -- right now. Here's what I've been up to, though:

I'm rearranging this so all the ETL is in standalone script files -- both .py and .R ("0-3_etl*") and the analysis is in analysis.Rmd file.

I also did some transforms so that all of the manual work would be in one step. The etl files ultimately output: [data/processed/2_etl_combined.xlsx](data/processed/2_etl_combined.xlsx). The ones that were coded as pertaining to pregnant patients are in [data/manual/confirmed_pregnant.xlsx](data/manual/confirmed_pregnant.xlsx), and the ones that were not are in [data/manual/not_confirmed_pregnant.xlsx](data/manual/not_confirmed_pregnant.xlsx) -- mostly generated from the following. Not sure where to put this code yet, should maybe leave it unpublished and only publish the resultant data sets.

In [2]:
import pandas

etl_2_combined = pandas.read_excel("data/processed/2_etl_combined.xlsx")
manual_pregnant_patient = pandas.read_excel("data/processed/EMTALA_PREGNANT_2011-2022--after_spotcheck.xlsx")
more_to_remove = ["2409 PTBH11","2409 D6YK11","2409 IT9611","2407 74BC11","2409 VRM911","2409 NZOV11","2406 05FK11","2407 ZY3N11","2406 JOU711","2406 4F4G11","2406 8E2Y11","2409 PKRO11","2407 T0DE11","2406 CITS12","2406 S4NX11","2406 RU2V11","2407 UD9X11","2407 KZUZ11","2406 D9FE11"]

pregnant_patient_keys = manual_pregnant_patient["key_identifier"].str.strip().unique().tolist()

not_confirmed_pregnant = etl_2_combined[~(etl_2_combined["key_identifier"].str.strip().isin(pregnant_patient_keys))|(etl_2_combined["key_identifier"].str.strip().isin(more_to_remove))|(etl_2_combined["hospital_type"]=="Psychiatric")]
not_confirmed_pregnant.to_excel("data/manual/not_confirmed_pregnant.xlsx",index=False)
print(not_confirmed_pregnant.shape)

confirmed_pregnant = etl_2_combined[(etl_2_combined["key_identifier"].str.strip().isin(pregnant_patient_keys))&(~etl_2_combined["key_identifier"].str.strip().isin(more_to_remove))&(etl_2_combined["hospital_type"]!="Psychiatric")]

turnaway = pandas.read_excel("data/processed/turnaway.xlsx")
turnaway["turnaway"] = turnaway["turnaway"].str.strip()
turnaway_keys = turnaway[turnaway["turnaway"].apply(lambda x: str(x)[0]=="Y")]["key_identifier"].unique()
confirmed_pregnant["turnaway"] = confirmed_pregnant["key_identifier"].isin(turnaway_keys)

confirmed_pregnant.to_excel("data/manual/confirmed_pregnant.xlsx",index=False)

(127, 19)


/var/folders/n1/qf74t1dj555cgs6f_mhz9nm0m6xrwl/T/ipykernel_6672/3019405527.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed_pregnant["turnaway"] = confirmed_pregnant["key_identifier"].isin(turnaway_keys)


I did spot checks of both of these files. I didn't find anything in the "confirmed_pregnant" file that pertained to non-pregnant patients, but I did find a few in the "not_confirmed_pregnant" file that appeared to pertain to pregnant patients.

#### I think there's a handful of cases that we're excluding that we might want to include -- log and signage violations.

This would change the actual numbers in our findings by a small amount. You can view them in [not_confirmed_pregnant--spotcheck.xlsx]("data/manual/not_confirmed_pregnant--spotcheck.xslx")

In [42]:
import pandas 

spotcheck = pandas.read_excel("data/manual/not_confirmed_pregnant--spotcheck.xlsx",index_col=0)
spotcheck[spotcheck["pregnant?"]=="yes"][["key_identifier","notes"]]

,key_identifier,notes
Unnamed: 0,,
68,2402 YOYJ11,it's a signage thing that definitely affected ...
660,2402 ZSG311,it's a signage thing that definitely affected ...
523,2405 78J911,it's a log keeping thing that definitely affec...
169,2402 WEKB11,it's a signage thing that definitely affected ...
517,2405 724D11,it's a log keeping thing that definitely affec...
447,2405 X8E611,it's a log keeping thing that definitely affec...
522,2402 78J911,it's a signage policy thing that definitely af...
11,2402 C4HB11,it's a signage thing that definitely affected ...


#### There are a handful of "Y" responses that we're not counting among the turnaways

Most of them look like the turned-away patient was not actually pregnant, but at least two (pre-eclampsia, flagging as good insight) look like they were? This would also change the actual numbers in some of our findings by a small amount.

In [41]:
turnaway = pandas.read_excel("data/processed/turnaway.xlsx")
turnaway["turnaway"] = turnaway["turnaway"].str.strip()
turnaway[turnaway["turnaway"].apply(lambda x: str(x)[0]=="Y")].groupby("turnaway")["EVENT_ID"].nunique().sort_index()


turnaway
Y                                                        237
Y -- flagging as good insight into action by hospital      1
Y -- motor vehicle accident                                1
Y -- pre-eclampsia                                         1
Y -- same case same pregnant person                        1
Y -- same patient                                          1
Name: EVENT_ID, dtype: int64

#### I'm confused by what is happening in 3_etl_hosp_summary.R !

A large part of this is my R novitiate status -- I tried to clean up what was going on in R, but am not good enough at R to make it actually work ;^-^

When I do the analysis I _think_ you might be trying to do in Python, I'm getting different counts (Rachel found 135 rural, 203 non-rural)! The findings still hold true but we should resolve this before publishing the repo!

Web: “Scripps News manually reviewed hundreds of CMS investigations involving pregnant patients of both rural and urban hospitals.”

Script: “OUR INVESTIGATION FOUND THE CASES INVOLVE BOTH SMALL, RURAL FACILITIES..AND THOSE IN BIG CITIES”


In [80]:
rurban_hosp = pandas.read_excel("data/public/rural_urban_hospitals.xlsx")
rurban_hosp[rurban_hosp["facility_id"].duplicated()]
confirmed_pregnant["facility_id"] = confirmed_pregnant["facility_id"].astype(str)

hosp_summ = confirmed_pregnant.groupby(["facility_id","facility_name","city","state"]).agg({"EVENT_ID":"nunique","dfcncy_desc":list_unique,"inspection_date":list_unique}).reset_index()
emtala_hosp_rurban = hosp_summ.join(rurban_hosp.set_index("facility_id"),on="facility_id",rsuffix="_rurban")

inspections_by_rurban = emtala_hosp_rurban.groupby("Rural Status")["EVENT_ID"].sum()
hospitals_by_rurban = emtala_hosp_rurban.groupby("Rural Status")["facility_id"].nunique()
print("Total number of inspections finding violations in the time period (unique EVENT_ID), broken down by whether or not the hospital was rural:",inspections_by_rurban)
print("Total number of hospitals in violation in the time period (unique facility_id), broken down by whether or not the hospital was rural:",hospitals_by_rurban)




Total number of inspections finding violations in the time period (unique EVENT_ID), broken down by whether or not the hospital was rural: Rural Status
No     179
Yes    137
Name: EVENT_ID, dtype: int64
Total number of hospitals in violation in the time period (unique facility_id), broken down by whether or not the hospital was rural: Rural Status
No     165
Yes    128
Name: facility_id, dtype: int64


/var/folders/n1/qf74t1dj555cgs6f_mhz9nm0m6xrwl/T/ipykernel_11910/761598602.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed_pregnant["facility_id"] = confirmed_pregnant["facility_id"].astype(str)


### Pregnant women made up 15.3% of all EMTALA investigations in the time period -- not 16%.

My counts match Rachels so I think this just got rounded wrong.

```413 investigations of violations against pregnant patients, out of 2694 investigations of overall. ( 15.3303637713437 %)```


### I moved the keywords and stop phrases into their own files

I like including them in the README but they're taking up a lot of real estate, so I think it'd be better if we linked out to them.

I'm still going to refactor 1_etl_nearby_text_search.py to also load this data from files we can link to.

### Gonna do a quick copyedit of README.md and then we're done here!